In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from keras.layers import Dense,Input,Lambda
from keras.models import Model
import keras.backend as K

/home/elijahc/.pyenv/versions/3.5.2/envs/jupyterlab/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from src.models import build_dense,CachedModel
from src.utils import process_mnist,get_encoder,get_decoder, get_transcoder
from src.plot import example_results

In [3]:
(x_train,y_train), (x_test,y_test) = process_mnist()
input_shape=(784,)
print(x_train.shape)
print(x_test.shape)

(60000, 784)
(10000, 784)


In [4]:
# Lets build a vanilla autoencoder!
encoding_dim = 32
inputs = Input(shape=input_shape)
encoded = build_dense(inputs,[128,64,encoding_dim],activations='relu')
decoded = build_dense(encoded,[64,128,784],activations=['relu','relu','sigmoid'])
ae_mod = Model(inputs,decoded)
ae_mod.outputs
# autoencoder = LazyModel(ae_mod,name='AE_3_layer')
# autoencoder.compile(optimizer='adadelta',loss='binary_crossentropy')

[<tf.Tensor 'dense_6/Sigmoid:0' shape=(?, 784) dtype=float32>]

In [5]:
autoencoder = CachedModel(inputs,decoded,name='AE_3_layer',verbose=1)

Loading Cached Model: models/2018-04-07/AE_3_layer


In [6]:
autoencoder.compile(optimizer='adadelta',loss='binary_crossentropy')

Cached to: 
 /home/elijahc/projects/vae/models/2018-04-07/AE_3_layer/model.h5


In [7]:
encoder=get_encoder(autoencoder)
encoded_input = Input(shape=(encoding_dim,))
decoder=get_decoder(autoencoder,encoded_input)

In [8]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_5 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_6 (Dense)              (None, 784)               101136    
Total para

In [9]:
autoencoder.fit(x_train,x_train,
               epochs=50,
               batch_size=256,
               shuffle=True,
               validation_data=(x_test,x_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 3s 49us/step - loss: 0.3356 - val_loss: 0.2631
Epoch 2/50
60000/60000 [==============================] - 3s 45us/step - loss: 0.2518 - val_loss: 0.2440
Epoch 3/50
60000/60000 [==============================] - 3s 45us/step - loss: 0.2380 - val_loss: 0.2282
Epoch 4/50
60000/60000 [==============================] - 3s 45us/step - loss: 0.2217 - val_loss: 0.2109
Epoch 5/50
60000/60000 [==============================] - 3s 45us/step - loss: 0.2039 - val_loss: 0.1946
Epoch 6/50
60000/60000 [==============================] - 3s 45us/step - loss: 0.1905 - val_loss: 0.1858
Epoch 7/50
60000/60000 [==============================] - 3s 45us/step - loss: 0.1839 - val_loss: 0.1801
Epoch 8/50
60000/60000 [==============================] - 3s 45us/step - loss: 0.1790 - val_loss: 0.1776
Epoch 9/50
60000/60000 [==============================] - 3s 45us/step - loss: 0.1749 - val_loss: 0.1717
Epoch

In [10]:
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

In [11]:
example_results(x_test,encoder,decoder,shuffle=True)

NameError: name 'transcoder' is not defined